In [3]:
import plotly
plotly.__version__
import pandas as pd

In [4]:
mdata = pd.read_csv("DataSets/belly_button_metadata.csv")
mdata.head()

,sample,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [5]:
bbdata = pd.read_csv("Datasets/belly_button_data.csv")
bbdata.head()

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.orm import load_only
from sqlalchemy import func, desc

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [14]:
engine = create_engine("sqlite:///db/belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [15]:
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [16]:
Samples_Metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

In [17]:
session = Session(engine)

In [112]:
def idCollectorPC (sample):
    fields = [sample]
    results = session.query(Samples).options(load_only(sample)).order_by(desc(sample))
    
    collector = []
    names = []
    values = []
    
    for result in results[:10]:
        row = result.__dict__    
        names.append(f"Otu - {row['otu_id']}")
        values.append(row[f"{sample}"])
    
    values = [round((x/sum(values))*100,2) for x in values]
        
    return {
        'Name':names,
        'values':values
    }
# function is functioning


In [113]:
idCollectorPC('BB_960')

{'Name': ['Otu - 922',
  'Otu - 944',
  'Otu - 1795',
  'Otu - 2419',
  'Otu - 943',
  'Otu - 2722',
  'Otu - 1929',
  'Otu - 3541',
  'Otu - 976',
  'Otu - 1014'],
 'values': [24.54, 20.86, 19.63, 16.56, 3.68, 3.68, 3.07, 3.07, 2.45, 2.45]}

In [ ]:
def idCollectorPC_f(sample):
    all_otus = session.query(Otu).statement
    all_otus_df = pd.read_sql_query(all_otus, session.bind)
    all_otus_df.set_index('otu_id', inplace=True)
    
    return jsonify(list(all_otus_df["lowest_taxonomic_unit_found"]))

In [ ]:
idCollectorPC_f('BB_960')

In [ ]:
def metadata(sample):
    all_samples_metadata = session.query(Samples_Metadata).statement
    all_samples_metadata_df = pd.read_sql_query(all_samples_metadata, session.bind)
    sample_num = int(sample.split("_")[1])
    selected_sample = all_samples_metadata_df.loc[all_samples_metadata_df["SAMPLEID"] == sample_num, :]
    json_selected_sample = selected_sample.to_json(orient='records')
    return json_selected_sample

In [ ]:
metadata('BB_960')

In [ ]:
def subjectCollectorBG(sample_number):
    info_id = sample_number[3:]
    results = session.query(Samples_Metadata).filter(Samples_Metadata.SAMPLEID == info_id).first()
    
    print(results)
    
    return {
        'age':results.AGE,
        'gender':results.GENDER,
        'ethnicity':results.ETHNICITY,
        'source':results.EVENT
    }

In [ ]:
def OtuCollectorBC():
    results = session.query(Samples).all()
    x = []
    y = []
    
    for result in results:
        row = result.__dict__
        otu_values = list(row.values())
        x.append(result.otu_id)
        y.append(sum(otu_values[1:]))
        
    return {
        'x':x,
        'y':y
    }
# function is functioning      

In [ ]:
OtuCollectorBC()

In [28]:
def idListN():
    columns = Samples.__table__.columns.keys()
    return columns[1:]
# function is functioning

In [32]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.orm import load_only
from sqlalchemy import func, desc

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

engine = create_engine("sqlite:///db/belly_button_biodiversity.sqlite")
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [35]:
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [36]:
Samples_Metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

In [37]:
session = Session(engine)

In [40]:
def SampleNamesCollector():
    sampleNames = []
    for sampleid in session.query(Samples_Metadata.SAMPLEID).all():
        sampleNames.append("BB_"+str(sampleid[0]))
    return sampleNames

In [41]:
SampleNamesCollector()

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 

In [104]:
def OTUbySamplesCollector(sample_id):
    # assign the db's uniform resource identifier to a variable
    db_uri = 'sqlite:///db/belly_button_biodiversity.sqlite'

    #connect to the db using create_engine():
    from sqlalchemy import create_engine
    # returned value engine represents the core interface to the database
    engine = create_engine(db_uri,echo=False)

    # Declare a Base using `automap_base()`
    from sqlalchemy.ext.automap import automap_base
    Base = automap_base()

    # Use the Base class to reflect the database tables
    Base.prepare(engine, reflect=True)

    # mapped classes are now created with names by default
    # matching that of the table name.
    Otu = Base.classes.otu
    Samples = Base.classes.samples
    Samples_Metadata = Base.classes.samples_metadata

    # Create a session
    from sqlalchemy.orm import Session
    session = Session(engine)

    # initialize an empty list to store the sample table
    otuIDbySample = []

    for row in session.query(Samples).all():
        otuIDbySample.append(row.__dict__)

    import pandas as pd
    otuIDbySample = pd.DataFrame.from_dict(otuIDbySample, orient='columns', dtype=None)

    otuIDbySample = otuIDbySample[sample_id].sort_values(ascending=False)[:10].reset_index()
    otuIDbySample.columns =["otu_ids","sample_values"]

    return otuIDbySample.to_json(orient='columns')

In [105]:
OTUbySamplesCollector('BB_940')

'{"otu_ids":{"0":1166,"1":2858,"2":481,"3":2263,"4":40,"5":1188,"6":351,"7":188,"8":2317,"9":1976},"sample_values":{"0":163,"1":126,"2":113,"3":78,"4":71,"5":51,"6":50,"7":47,"8":40,"9":40}}'

In [69]:
def SampleMetaDataCollector(sample_id):
    # assign the db's uniform resource identifier to a variable
    db_uri = 'sqlite:///db/belly_button_biodiversity.sqlite'

    #connect to the db using create_engine():
    from sqlalchemy import create_engine
    # returned value engine represents the core interface to the database
    engine = create_engine(db_uri,echo=False)

    # Declare a Base using `automap_base()`
    from sqlalchemy.ext.automap import automap_base
    Base = automap_base()

    # Use the Base class to reflect the database tables
    Base.prepare(engine, reflect=True)

    # mapped classes are now created with names by default
    # matching that of the table name.
    Otu = Base.classes.otu
    Samples = Base.classes.samples
    Samples_Metadata = Base.classes.samples_metadata

    # Create a session
    from sqlalchemy.orm import Session
    session = Session(engine)

    # initialize an empty list to store the sample metadata table
    sampleMetaData = []

    for row in session.query(Samples_Metadata).all():
        sampleMetaData.append(row.__dict__)

    import pandas as pd
    sampleMetaData = pd.DataFrame.from_dict(sampleMetaData, orient='columns', dtype=None)

    sample_id = pd.to_numeric(sample_id[3:])

    print(sample_id)
    meta  = sampleMetaData[sampleMetaData['SAMPLEID']==sample_id]
    meta = meta[['AGE','BBTYPE','ETHNICITY','GENDER','LOCATION','SAMPLEID']]

    meta = [{u: str(v)} for (u, v) in meta.iloc[0].iteritems()]
    return meta

In [70]:
 SampleMetaDataCollector('BB_960')

960


[{'AGE': '48'},
 {'BBTYPE': 'I'},
 {'ETHNICITY': 'Caucasian'},
 {'GENDER': 'M'},
 {'LOCATION': 'Lexington/NC'},
 {'SAMPLEID': '960'}]

In [127]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/belly_button_biodiversity.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table



/Users/mgarza/anaconda3/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning:

SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.



In [128]:
Base.classes.keys()


['otu', 'samples', 'samples_metadata']

In [129]:
Samples_Metadata = Base.classes.samples_metadata
Samples = Base.classes.samples

In [130]:
sample_metadata()

TypeError: sample_metadata() missing 1 required positional argument: 'sample'